# Filter female wavs

In [1]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import isfile, join
import os
import scipy 
import pandas as pd
import shutil

In [2]:
df=pd.read_csv(r"C:\Users\Tasli\Desktop\PythonFolder\SpeechEmotionClassifier\cremad\VideoDemographics.csv")
df.head(3)

,ActorID,Age,Sex,Race,Ethnicity
0,1001,51,Male,Caucasian,Not Hispanic
1,1002,21,Female,Caucasian,Not Hispanic
2,1003,21,Female,Caucasian,Not Hispanic


In [4]:
female_ids=[str(df.loc[i].iat[0]) for i in df.index if df.loc[i].iat[2]=="Female"]

In [5]:
audio_folder_name=r'C:\Users\Tasli\Desktop\PythonFolder\SpeechEmotionClassifier\cremad\AudioWAV'
dest_folder_name= r'C:\Users\Tasli\Desktop\PythonFolder\SpeechEmotionClassifier\cremad\FemaleAudioWAV'
file_names = listdir(audio_folder_name)
for f in file_names:
    
    x=f.split("_")
    if x[0] not in female_ids:
       continue
    #print(f)
    shutil.move(audio_folder_name + '\\' +f, dest_folder_name)
    

# Convert to mel specs

In [46]:
##FUNCTIONS

def convert_single_audio_to_image(audio_path, image_path, spectrogram_dimensions = (64,64)):
    data, sr = librosa.load(audio_path, sr = 22050)  
    print("sr={}".format(sr))
    #Make a mel spectrogram from audio
    window_width =  0.025 #25 ms  window size 
    sliding = 0.01 #10ms stride 
    print("Length of audio:", np.size(data,0)/22050)
    spec = librosa.feature.melspectrogram(y=data, sr=sr, n_fft = int(window_width*sr), hop_length =int(sliding*sr))
    print(spec.shape)
    
    #Convert amplitude to decibels
    db_spec = librosa.power_to_db(spec, ref=np.max)
    print(db_spec.shape)
    fig = plt.figure()
    
    
   # fig.set_size_inches((spectrogram_dimensions[0]/fig.get_dpi(), spectrogram_dimensions[1]/fig.get_dpi()))
    
    #Display final mel spectrogram
    librosa.display.specshow(db_spec, sr=sr,x_axis='time',y_axis='mel', hop_length =int(sliding*sr))
    plt.colorbar()
    fig.savefig(image_path)
    plt.close(fig)


In [47]:
if __name__ == '__main__':
    audio_folder_name= r"C:\Users\Tasli\Desktop\PythonFolder\SpeechEmotionClassifier\cremad\AudioWAV\sam"
    image_folder_name=  r"C:\Users\Tasli\Desktop\PythonFolder\SpeechEmotionClassifier\cremad\AudioWAV\sam"
    file_names = listdir(audio_folder_name)
#     file_names = [f for f in listdir(audio_folder_name)]
    print("count of Wav_files in {}= {}".format(len(file_names),audio_folder_name))
    for file_name in file_names:
        name, ext = os.path.splitext(file_name)
        print(file_name)
            
        audio_path = audio_folder_name + '\\' + file_name
        spectogram_path = image_folder_name + '\\' + file_name.replace('.wav', '.png')
        convert_single_audio_to_image(audio_path,spectogram_path)
    image_files= listdir(image_folder_name)
    print("count of Wav_files in {}= {}".format(len(image_files),image_folder_name))

count of Wav_files in 1= C:\Users\Tasli\Desktop\PythonFolder\SpeechEmotionClassifier\cremad\AudioWAV\sam
1001_IEO_ANG_MD.wav
sr=22050
Length of audio: 2.635963718820862
(128, 3)
(128, 3)
count of Wav_files in 2= C:\Users\Tasli\Desktop\PythonFolder\SpeechEmotionClassifier\cremad\AudioWAV\sam
